### Prototypes

In [ ]:
'use strict';

When it comes to inheritance, JavaScript only has one construct: objects. Each object has a private property which holds a link to another object called its __prototype__.

That prototype object has a prototype of its own, and so on until an object is reached with `null` as its prototype.

Nearly all objects in JavaScript are instances of `Object`, which has `null` as its prototype.

Let's make a trip back to the zoo.

In [ ]:
class ZooError extends Error {

    // we are poor people, we don't have enum in JavaScript
    static ERROR_CODE = {
        ABSTRACT_INSTANCE_NOT_ALLOWED: 'ABSTRACT_INSTANCE_NOT_ALLOWED',
        METHOD_NOT_IMPLEMENTED: 'METHOD_NOT_IMPLEMENTED',
        ZOMBIE_NOT_ALLOWED: 'ZOMBIE_NOT_ALLOWED'
    };

    // this object has computed keys, instead of hardcoded strings
    static ERROR_MESSAGE = {
        [ZooError.ERROR_CODE.ABSTRACT_INSTANCE_NOT_ALLOWED]: 'You must extend this class',
        [ZooError.ERROR_CODE.METHOD_NOT_IMPLEMENTED]: 'You must implement this method',
        [ZooError.ERROR_CODE.ZOMBIE_NOT_ALLOWED]: 'This animal is too old to be alive'
    };

    constructor({ code, message = ZooError.ERROR_MESSAGE[code], data }) {
        super(message);

        this.code = code;
        this.data = data;
    }

    toString() {

        return JSON.stringify({
            code: this.code,
            message: this.message,
            data: this.data
        });
    }
}

class Animal {

    static maxAge = 100;

    // there is no such thing as protected constructor in JavaScript
    constructor({ birthDate = new Date(), name = 'unknown', species = 'unknown' } = {}) {

        // shower cabins are transparent in JavaScript, the _ is just a sticker that says "don't look"
        this._birthDate = birthDate;
        this._name = name;
        this._species = species;

        if (this.age > Animal.maxAge) {

            throw new ZooError({
                code: ZooError.ERROR_CODE.ZOMBIE_NOT_ALLOWED,
                data: {
                    species: this._species,
                    age: this.age,
                    maxAge: Animal.maxAge
                }
            });
        }
    }

    // this is a getter (for a made up property)
    get age() {

        const ageInMilliseconds = new Date() - this._birthDate
        return Math.floor(ageInMilliseconds / 1000 / 60 / 60 / 24 / 365); // convert to years
    }

    // we have to manually throw error for unimplemented method
    run() {

        throw new ZooError({
            code: ZooError.ERROR_CODE.METHOD_NOT_IMPLEMENTED,
            // methods are functions and all functions have the 'name' property
            data: { method: this.run.name }
        });
    }
}

class Dog extends Animal {
    
    constructor({ birthDate, name }) {
        super({ birthDate, name, species: 'dog' });
    }
    
    // implement required method
    run() {
        console.log('Du-te Dica!');
    }

    // override method
    greet() {
        console.log('Bark!');
        
        super.greet();
    }
}

In [ ]:
{
    const doggo = new Dog({ birthDate: new Date('01-01-2015'), name: 'Rex' });

    let proto = Object.getPrototypeOf(doggo);
    while (proto !== null) {

        console.log(proto.constructor.name);
        proto = Object.getPrototypeOf(proto);
    }
}

The private property we've mentioned at the beginning of the notebook is called __\_\_proto\_\___ in the most JavaScript engines, but the recommended way to access it is the one shown above __Object.getPrototypeOf()__, as we are given guarantess only for APIs.

In [ ]:
{
    const doggo = new Dog({ birthDate: new Date('01-01-2015'), name: 'Rex' });

    let proto = doggo.__proto__;
    while (proto !== null) {

        console.log(proto.constructor.name);
        proto = proto.__proto__;
    }
}

Let's see what's actually behind the `class` syntactic sugar. It's just a function whose name is in PascalCase (a convention, like the _ for private properties).

In [ ]:
function ExampleOfOldSchoolClass(name) {
    this._name = name;
    this._count = 0;
}

The convention instructs us that this function can be used as a constructor (using the `new` operator) to create objects whose __\_\_proto\_\___ property will be equal to this function's __prototype__ property.

Wait .. what? 

Let's focus just on the constructor part for the moment.

In [ ]:
{
    const instance = new ExampleOfOldSchoolClass('Ionel');
    
    let proto = instance.__proto__;
    while (proto !== null) {
        
        console.log(proto.constructor.name);
        proto = proto.__proto__;
    }
}

Ok, we have a constructor for our class, but our instance has only some 'private' properties. What about methods?

Here comes the function's __prototype__ property into the game. 

In JavaScript a function is an object like any other object, with the additional swag that is callable. 

The __prototype__ property on this object / function, though, has a special meaning - it means that all instances created with this function will have the private __\_\_proto\_\___ property equal to __MyFunction.prototype__.

Let's add a method in our 'class' to better understand the whole proto-spaghetti dish.

In [ ]:
console.log(ExampleOfOldSchoolClass.prototype);

In [ ]:
ExampleOfOldSchoolClass.prototype.increment = function (by) {
    this._count += by;
}
console.log(ExampleOfOldSchoolClass.prototype);

In [ ]:
{
    const instance = new ExampleOfOldSchoolClass('Ionel');

    console.log(instance.__proto__.constructor.name);
    console.log(instance.__proto__);

    instance.increment(3);
    console.log(instance._count);
}